In [20]:
import pandas as pd
import xml.etree.ElementTree as ET
import geopandas
import numpy as np
from shapely.geometry import Point
import json

In [21]:
ET.register_namespace('', 'http://www.w3schools.com')
namespace = {'xmlns': 'http://www.w3schools.com'}
tree = ET.parse('data/sjobo_initial_solution.xml')

In [22]:
with open("data/population_sjobo_others_only_sjobo.json") as f:
    j = json.load(f)

In [23]:
persons = []
for person in j['persons']:
    slat = person['activities'][0]['coord']['lat']
    slon = person['activities'][0]['coord']['lon']
    elat = person['activities'][1]['coord']['lat']
    elon = person['activities'][1]['coord']['lon']
    
    persons.append({
        'id': person['id'],
        'start_longitude': slon,
        'start_latitude': slat,
        'end_longitude': elon,
        'end_latitude': elat,
        'type': person['activities'][1]['type'],
        'time': person['activities'][1]['start_time'],
        'szone': person['activities'][0]['zone'],
        'ezone': person['activities'][1]['zone'],
    })
persons = pd.DataFrame(persons)

In [24]:
person

{'activities': [{'coord': {'lat': 55.64946746826172,
    'lon': 13.898370742797852},
   'end_time': '55200',
   'start_time': 0,
   'type': '2',
   'zone': 1265},
  {'coord': {'lat': 55.67042647575592, 'lon': 13.915060310589341},
   'end_time': 86400,
   'start_time': '55200',
   'type': '1',
   'zone': 1265}],
 'attributes': {'age': 'Ald65_W'},
 'id': 3310948}

In [25]:
persons

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone
0,386,12.569857,56.161778,13.758286,55.690662,2,14760,1284,1265
1,387,13.758286,55.690662,12.569857,56.161778,1,55920,1265,1284
2,3905,12.561804,56.180489,13.696956,55.631886,2,24900,1284,1265
3,3906,13.696956,55.631886,13.493143,55.634171,16,38040,1265,1281
4,3908,14.213787,55.844013,13.696956,55.631886,2,27840,1290,1265
...,...,...,...,...,...,...,...,...,...
68984,3310944,13.898371,55.649467,13.911258,55.667062,1,67500,1265,1265
68985,3310945,13.915060,55.670426,13.891012,55.655105,4,47460,1265,1265
68986,3310946,13.891012,55.655105,13.889977,55.653408,6,64740,1265,1265
68987,3310947,13.889977,55.653408,13.898371,55.649467,11,62820,1265,1265


In [26]:
persons.to_excel('data/sjöbo_trips_others.xlsx', index=False)

# run "IVT between points with OSRM" 

In [27]:
direct_time = pd.read_excel('data/direct_time.xlsx')

In [28]:
persons['direct_time'] = direct_time.direct_time

In [29]:
persons

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone,direct_time
0,386,12.569857,56.161778,13.758286,55.690662,2,14760,1284,1265,5898.5
1,387,13.758286,55.690662,12.569857,56.161778,1,55920,1265,1284,5826.8
2,3905,12.561804,56.180489,13.696956,55.631886,2,24900,1284,1265,5329.9
3,3906,13.696956,55.631886,13.493143,55.634171,16,38040,1265,1281,887.5
4,3908,14.213787,55.844013,13.696956,55.631886,2,27840,1290,1265,3282.6
...,...,...,...,...,...,...,...,...,...,...
68984,3310944,13.898371,55.649467,13.911258,55.667062,1,67500,1265,1265,230.6
68985,3310945,13.915060,55.670426,13.891012,55.655105,4,47460,1265,1265,304.6
68986,3310946,13.891012,55.655105,13.889977,55.653408,6,64740,1265,1265,50.8
68987,3310947,13.889977,55.653408,13.898371,55.649467,11,62820,1265,1265,173.5


In [30]:
max_ivt = []
for person in persons.itertuples():
#     if person.szone == 1265 and person.ezone == 1265:
#         max_ivt.append(person.direct_time * 1.7)
#     elif person.szone == 1265 and person.ezone != 1265:
#         max_ivt.append(person.direct_time * 1.95)
#     elif person.szone != 1265 and person.ezone == 1265:
#         max_ivt.append(person.direct_time * 1.55)
    max_ivt.append(person.direct_time * 1.5)

In [31]:
persons['max_ivt'] = max_ivt

In [32]:
persons = persons[((persons.szone == 1265) | (persons.ezone == 1265)) & (persons.direct_time > 120)]

In [33]:
sample = persons.sample(int(len(persons.index)*0.1))

In [34]:
sample

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone,direct_time,max_ivt
48146,2927228,13.919950,55.608237,13.938777,55.618972,6,60120,1265,1270,215.9,323.85
32548,2294487,13.644497,55.688159,13.706905,55.719488,3,57600,1265,1265,675.4,1013.10
1812,662095,13.789005,55.700407,13.020321,55.604502,1,71880,1265,1280,3148.7,4723.05
20319,2281444,13.648706,55.633399,13.690241,55.631648,3,44040,1265,1265,316.8,475.20
51592,3092709,13.537527,55.679135,13.557877,55.607092,2,25380,1281,1265,1134.4,1701.60
...,...,...,...,...,...,...,...,...,...,...,...
22651,2283923,13.637871,55.634063,13.693960,55.570629,2,31320,1265,1265,975.1,1462.65
59516,3132064,13.836564,55.683397,13.769772,55.626500,3,45120,1265,1265,906.8,1360.20
8045,1392688,13.643779,55.689657,13.189475,55.698330,1,63120,1265,1281,2211.3,3316.95
37726,2327930,13.832852,55.431484,13.705436,55.631752,2,36960,1286,1265,1667.0,2500.50


In [35]:
sample.to_excel('data/sjöbo_trips_general_sample6421.xlsx', index=False)

In [18]:
persons.to_excel('data/sjöbo_trips_others.xlsx', index=False)

In [14]:
# persons.replace(0, np.NaN).dropna().sample(1500).to_excel('data/sjöbo_trips_others_sample1500.xlsx', index=False)